In [1]:
from datetime import datetime, date

import numpy
from matplotlib import pyplot
import pandas
import seaborn
import altair

import ipywidgets

import cvd
seaborn.set(style='ticks', context='paper', palette='deep')
%matplotlib inline

In [4]:
data = pandas.concat(
    map(cvd.read_data, ['Confirmed', 'Recovered', 'Deaths']),
    axis='columns'
).reset_index().assign(Subset=lambda df: numpy.select(
    (df['Country/Region'].isin(('US', 'Canada')), df['Country/Region'] == 'China'),
    ('US/Canada', 'China'), 'Everywhere\nElse'
))

cumxtab = cvd.cumulative_xtab(data, 'Subset')
us_data = cvd.get_us_state_data(data)

,Province/State,Date,Confirmed,Recovered,Deaths,State
13664,"Adams, IN",2020-01-22,0,0,0,IN
13665,"Adams, IN",2020-01-23,0,0,0,IN
13666,"Adams, IN",2020-01-24,0,0,0,IN
13667,"Adams, IN",2020-01-25,0,0,0,IN
13668,"Adams, IN",2020-01-26,0,0,0,IN


In [7]:
dates = us_data['Date'].unique()
options = [pandas.Timestamp(d).strftime('%m/%d') for d in dates]

@ipywidgets.interact(
    us_data=ipywidgets.fixed(us_data),
    state=ipywidgets.Dropdown(
        options=cvd._state_abbrev.items(),
        value='US',
        description='Pick a State:',
        disabled=False
    ),
    dates=ipywidgets.SelectionRangeSlider(
        index=(0, len(dates)-1),
        options=options,
        description='Dates',
        disabled=False,
        layout=ipywidgets.Layout(width='85%'),
        continuous_update=False
    )
)
def state_chart(us_data, state, dates):
    if state == 'US':
        data = cvd._us_data(us_data, dates)
    else:
        data = cvd._state_data(us_data, state, dates)
    return cvd._area_chart(data)

interactive(children=(Dropdown(description='Pick a State:', options=(('United States', 'US'), ('Alabama', 'AL'…